In [2]:
import pandas as pd
#对50万样本随机采样3万
data = pd.read_csv("/data/fjsdata/mathV.csv",sep='|',low_memory=False)
data = data.fillna(0)#填补空值为零
print(data.shape)
#data = data.dropna(axis=0,how='any')
data=data.sample(n=100000)#随机采样5万
data.to_csv("/data/fjsdata/mathVR10W.csv",index=False,sep='|')
print(data.shape)

(542340, 3586)
(100000, 3586)


In [7]:
#贝叶斯多分类逻辑回归
import theano
import pymc3 as pm
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score,classification_report
import theano.tensor as tt
from sklearn import datasets
import time
import numpy as np
starttime = time.time()
#1.加载训练文件
data = pd.read_csv("/data/fjsdata/mathVR10W.csv",sep='|',low_memory=False)
print (data.shape)
#2.构造训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(data.drop(columns=['quid','qdiff']), data['qdiff'], test_size=.2)#8成训练，2成测试
#3.构建softmax regression model
X_input = theano.shared(X_train.values)#转numpy array
Y_output = theano.shared(Y_train.values)#转numpy array
with pm.Model() as EDP_model:
    beta = pm.Normal('beta', mu=0, sd=6, shape=(3584,6))
    alpha = pm.Normal('alfa', mu=0, sd=6, shape=6)
    mu = tt.dot(X_input,beta) + alpha
    p = pm.Deterministic('p', tt.nnet.softmax(mu))
    Y_obs = pm.Categorical('Y_obs', p=p, observed=Y_output)
#4.nuts采样
with EDP_model:
    start=pm.find_MAP()  # 参数初猜
    step = pm.Metropolis()#多分类采样#step = pm.NUTS()连续采样
    trace = pm.sample(1000,start=start,step=step,chains=2,cores=2)
    #pm.traceplot(trace)
X_input.set_value(X_test.values)#测试集
Y_output.set_value(Y_test.values)#测试集
with EDP_model:
    ppc = pm.sample_posterior_predictive(trace)#后验预测 
    Y_pred = ppc['Y_obs'].mean(axis=0)
print ('测试集准确率：%f'% accuracy_score(Y_test,Y_pred.round()))
print (classification_report(Y_test,Y_pred.round()))
endtime = time.time()
print ("Complete time: %f s" % (endtime - starttime))

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


(100000, 3586)


/usr/local/lib/python3.6/dist-packages/pymc3/tuning/starting.py:61: UserWarning: find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.
  warnings.warn('find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.')
logp = -86,720, ||grad|| = 2.9813: 100%|██████████| 930/930 [43:18<00:00,  2.79s/it]          
Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>Metropolis: [alfa]
>Metropolis: [beta]
Sampling 2 chains: 100%|██████████| 3000/3000 [1:28:47<00:00,  3.23s/draws]
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.
100%|██████████| 2000/2000 [43:12<00:00,  1.28s/it]


测试集准确率：0.838500
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     12997
        1.0       0.00      0.00      0.00        23
        2.0       0.00      0.00      0.00       930
        3.0       0.54      1.00      0.70      3788
        4.0       0.00      0.00      0.00      2235
        5.0       0.00      0.00      0.00        27

avg / total       0.75      0.84      0.78     20000

Complete time: 10896.869679 s


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [3]:
#多分类逻辑回归
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
import time
start = time.time()
#1.加载训练文件
data = pd.read_csv("/data/fjsdata/mathVR10W.csv",sep='|',low_memory=False)
print (data.shape)
#2.构造训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(data.drop(columns=['quid','qdiff']), data['qdiff'], test_size=.2)#8成训练，2成测试
#3.lr多分类
clf = LogisticRegression(penalty='l2',random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train,Y_train)
#4.预测
Y_pred= clf.predict(X_test)
print ('测试集准确率：%f'% accuracy_score(Y_test, Y_pred))
print (classification_report(Y_test,Y_pred))
end = time.time()
print ("Complete time: %f s" % (end - start))

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(100000, 3586)
测试集准确率：0.839450
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     13047
        1.0       0.00      0.00      0.00        28
        2.0       0.00      0.00      0.00       974
        3.0       0.54      1.00      0.70      3748
        4.0       0.00      0.00      0.00      2174
        5.0       0.00      0.00      0.00        29

avg / total       0.75      0.84      0.78     20000

Complete time: 404.423762 s


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [4]:
#多分类神经网络
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score,classification_report
import time
start = time.time()
#1.加载训练文件
data = pd.read_csv("/data/fjsdata/mathVR10W.csv",sep='|',low_memory=False)
print (data.shape)
#2.构造训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(data.drop(columns=['quid','qdiff']), data['qdiff'], test_size=.2)#8成训练，2成测试
#3.多分类
clf = MLPClassifier(hidden_layer_sizes=(100,100,100),solver='sgd').fit(X_train,Y_train)
#4.预测
Y_pred= clf.predict(X_test)
print ('测试集准确率：%f'% accuracy_score(Y_test, Y_pred))
print (classification_report(Y_test,Y_pred))
end = time.time()
print ("Complete time: %f s" % (end - start))

(100000, 3586)
测试集准确率：0.837250
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     12974
        1.0       0.00      0.00      0.00        31
        2.0       0.00      0.00      0.00       955
        3.0       0.54      1.00      0.70      3773
        4.0       0.00      0.00      0.00      2250
        5.0       0.00      0.00      0.00        17

avg / total       0.75      0.84      0.78     20000

Complete time: 413.102002 s


In [5]:
#决策树分类
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,classification_report
import time
start = time.time()
#1.加载训练文件
data = pd.read_csv("/data/fjsdata/mathVR10W.csv",sep='|',low_memory=False)
print (data.shape)
#2.构造训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(data.drop(columns=['quid','qdiff']), data['qdiff'], test_size=.2)#8成训练，2成测试
#3.决策树分类
clf = DecisionTreeClassifier().fit(X_train,Y_train)
#4.预测
Y_pred= clf.predict(X_test)
print ('测试集准确率：%f'% accuracy_score(Y_test, Y_pred))
print (classification_report(Y_test,Y_pred))
end = time.time()
print ("Complete time: %f s" % (end - start))

(100000, 3586)
测试集准确率：0.838600
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     12909
        1.0       0.00      0.00      0.00        19
        2.0       0.00      0.00      0.00      1004
        3.0       0.55      1.00      0.71      3871
        4.0       0.00      0.00      0.00      2175
        5.0       0.00      0.00      0.00        22

avg / total       0.75      0.84      0.78     20000

Complete time: 326.816416 s


In [6]:
#LGBM集成学习分类
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score,classification_report
import time
start = time.time()
#1.加载训练文件
data = pd.read_csv("/data/fjsdata/mathVR10W.csv",sep='|',low_memory=False)
print (data.shape)
#2.构造训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(data.drop(columns=['quid','qdiff']), data['qdiff'], test_size=.2)#8成训练，2成测试
train_data=lgb.Dataset(X_train,label=Y_train)
validation_data=lgb.Dataset(X_test,label=Y_test)
#3.LGBM集成学习分类
params={'learning_rate':0.1,
        'lambda_l1':0.1,
        'lambda_l2':0.2,
        'max_depth':4,
        'objective':'multiclass',
        'num_class':6}
clf=lgb.train(params,train_data,valid_sets=[validation_data])
#4.预测
Y_pred= clf.predict(X_test)
Y_pred=[list(x).index(max(x)) for x in Y_pred]
print ('测试集准确率：%f'% accuracy_score(Y_test, Y_pred))
print (classification_report(Y_test,Y_pred))
end = time.time()
print ("Complete time: %f s" % (end - start))

(100000, 3586)
[1]	valid_0's multi_logloss: 0.920791
[2]	valid_0's multi_logloss: 0.853038
[3]	valid_0's multi_logloss: 0.796912
[4]	valid_0's multi_logloss: 0.749453
[5]	valid_0's multi_logloss: 0.708744
[6]	valid_0's multi_logloss: 0.673449
[7]	valid_0's multi_logloss: 0.642601
[8]	valid_0's multi_logloss: 0.615458
[9]	valid_0's multi_logloss: 0.591449
[10]	valid_0's multi_logloss: 0.570116
[11]	valid_0's multi_logloss: 0.551089
[12]	valid_0's multi_logloss: 0.53406
[13]	valid_0's multi_logloss: 0.518781
[14]	valid_0's multi_logloss: 0.505032
[15]	valid_0's multi_logloss: 0.492634
[16]	valid_0's multi_logloss: 0.481426
[17]	valid_0's multi_logloss: 0.471287
[18]	valid_0's multi_logloss: 0.462083
[19]	valid_0's multi_logloss: 0.453726
[20]	valid_0's multi_logloss: 0.446119
[21]	valid_0's multi_logloss: 0.439196
[22]	valid_0's multi_logloss: 0.432868
[23]	valid_0's multi_logloss: 0.427088
[24]	valid_0's multi_logloss: 0.421813
[25]	valid_0's multi_logloss: 0.416978
[26]	valid_0's multi